# KR260 HLS QPSK Demod

## Setup

### Mounting Development Machine
This notebook utilizes `sshfs` to mount the development directory on the KR260 board to avoid transferring files bewteen machines.  

The development machine that was used to build the KR260 FPGA image is named `dev-wks`.  To mount the development directory on `dev-wks`, first will need to create a folder to mount the directory, then execute the following from the Jupyter Terminal:

```bash
cd /home/root/jupyter_notebooks
mkdir puch
sshfs sdr@dev-wks:/home/sdr/workspace/puch-workspace/HLS-QPSK-Demod-Baseband_002 /home/root/jupyter_notebooks/puch
```

If `sshfs` is not installed, then execute `sudo apt install sshfs` on the Jupyter Terminal.

### Unmounting Development Machine
To unmount the directory
` fusermount -u /home/root/jupyter_notebooks/dev-wks/`


### Plot function for use in this notebook
The first code block below defines a function that we will use for plotting data throughout this notebook. Note that the function has a `n_samples` argument so that we can limit the number of samples to plot. Plotting more than a few thousand samples can be very slow and consume a lot of RAM.

### Requirements
Install the following:

```bash
pip install numpy==1.26.4
pip install ipympl
pip install ipywidgets
pip install jupyter_bokeh
```

# Initilize

#### Bokeh Plot Function

In [ ]:
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook, show
import numpy as np
import math


def plot_time(in_signal,out_signal=None):
    t = np.linspace(0,len(in_signal),len(in_signal))
    output_notebook()
    p = figure(title = "Input & Output Signal")
    
    if out_signal is not None:
        p.line(t,out_signal,legend_label="Output Signal",line_color="red",line_width=1)
    p.line(t,in_signal,legend_label="Input Signal",line_color="blue",line_width=3)
    show(p)

#### Load Overlay

In [ ]:
from pynq import Overlay
from pynq import allocate
from pynq import MMIO
import pynq.lib.dma
import time

#import Python library from repo:
import sys
sys.path.append('/root/jupyter_notebooks/puch/')
import fpga.py.puch as puch
import fpga.lib.timestamp.sw.timestamp_regmap as timestamp_regmap
import fpga.lib.led_reg.sw.led_regmap as led_regmap
import fpga.lib.HLS_QPSK_Demod.sw.qpsk_regmap as qpsk_regmap
import fpga.lib.AWGN_GNG.sw.awgn_regmap as awgn_regmap
import fpga.lib.DMA_Data_Capture.sw.dma_data_capture_regmap as dma_capture_regmap

# Load the overlay
overlay = Overlay('../../overlays/KR260_HLS_QPSK_Demod/output/kr260_hls_qpsk_demod.bit')

# Assign blocks to short vars
dma          = overlay.axi_dma_0
led_module   = overlay.led_reg_0
timestamp    = overlay.Timestamp_0
qpsk_demod   = overlay.QPSK_Demod_Top_0
awgn         = overlay.gng_top_0 
dma_capture  = overlay.DMA_Data_Capture_Top_0

# Configure QPSK Syncword, and reset sync lock
#qpsk_demod.mmio.write(qpsk_regmap.RegMap.SYNC_WORD_ADDR,0xDEADBEEF)
qpsk_demod.mmio.write(qpsk_regmap.RegMap.SYNC_WORD_ADDR,0x6D75521E)
qpsk_demod.mmio.write(qpsk_regmap.RegMap.SYNC_RESET_ADDR,1)

# Read data path formats
fin_tBits, fin_fBits = puch.get_format(qpsk_demod,qpsk_regmap.RegMap.F_IN_ADDR)
fout_tBits, fout_fBits = puch.get_format(qpsk_demod,qpsk_regmap.RegMap.F_OUT_ADDR)
awgn_tBits, awgn_fBits = puch.get_format(awgn,awgn_regmap.RegMap.F_AWGN_ADDR)

print("Done Init System")

##### Display Time Stamp Register
The time stamp is burned into the FPGA during the build process

In [ ]:
print("FPGA Build Timestamp:  " + puch.get_timestamp_str(timestamp))

##### Toggle USER_LED[1:0] on/off

In [ ]:
for i in range(10):
    led_module.mmio.write(led_regmap.RegMap.USER_LEDS_ADDR,0x1)
    time.sleep(0.25)
    led_module.mmio.write(led_regmap.RegMap.USER_LEDS_ADDR,0x2)
    time.sleep(0.25)
led_module.mmio.write(led_regmap.RegMap.USER_LEDS_ADDR,0x0)

### Open Modulated Data Samples and Convert to INT16 I/Q Samples

In [ ]:
mod_samps = []
with open('../../lib/HLS_QPSK_Demod/sim/0xDEADBEEF_Tx_Samps.dat') as mod:
    mod_samps = mod.readlines()
mod_samps = [line.rstrip('\n') for line in mod_samps]

mod_samps = np.array(mod_samps)
mod_samps = np.concatenate((mod_samps, np.zeros(256))) # Add zeros to push through MF FIR
mod_samps = mod_samps.astype('float64')*(2**fin_fBits)
mod_samps = mod_samps.astype(np.int32)
xi = mod_samps[0::2]
xq = mod_samps[1::2]
xiq = (xi<<16) + xq # pack I/Q 32-bit

print("Number of I Samples Read: " + str(len(xi)))
print("Number of Q Samples Read: " + str(len(xq)))
print("Number Symbols          : " + str(len(xq)/16))
plot_time(xi)
#print(hex(xi[1:10]))

### DMA Transfer the Sample buffer from ARM Processor to the HLS QPSK Demodulator

In [43]:
# 16 Samples Per Symbol, therefore create read buffer /16
SamplesPerSym     = 16
NumSamples        = len(xiq)
NumSymbols        = int(NumSamples/SamplesPerSym)
AWGN_ENABLE       = 0 # 0: Disabled, 1:Enabled
awgn_gain         = 0.8
CAP_Symbols       = 32

print("NumSamples                    : " + str(NumSamples))
print("NumSymboles                   : " + str(NumSymbols))

#  AWGN 
awgn.write(awgn_regmap.RegMap.AWGN_NOISE_GAIN_ADDR,int(awgn_gain*(2**awgn_fBits)))
awgn.write(awgn_regmap.RegMap.AWGN_ENABLE_ADDR,AWGN_ENABLE)
print("AWGN Enable                   : " + str(awgn.read(awgn_regmap.RegMap.AWGN_ENABLE_ADDR)))
print("AWGN Gain                     : " + str(awgn.read(awgn_regmap.RegMap.AWGN_NOISE_GAIN_ADDR)/(2**awgn_fBits)))


# Reset DMA Capture Buffer and configure a capture

dma_capture.mmio.write(dma_capture_regmap.RegMap.CAPTURE_LENGTH_ADDR, CAP_Symbols)
dma_capture.mmio.write(dma_capture_regmap.RegMap.FIFO_FLUSH_ADDR,1)

print("DMA Reset                     : " + str(dma_capture.mmio.read(dma_capture_regmap.RegMap.FIFO_FLUSH_ADDR)))
dma_capture.mmio.write(dma_capture_regmap.RegMap.FIFO_FLUSH_ADDR,0)
print("DMA Reset                     : " + str(dma_capture.mmio.read(dma_capture_regmap.RegMap.FIFO_FLUSH_ADDR)))

# Configure DMA Capture Buffer
print("DMA FIFO Depth                : " + str(dma_capture.mmio.read(dma_capture_regmap.RegMap.MAX_DEPTH_ADDR)))
print("DMA FIFO Capture Length       : " + str(dma_capture.mmio.read(dma_capture_regmap.RegMap.CAPTURE_LENGTH_ADDR)))
print("DMA FIFO Write Pointer        : " + str(dma_capture.mmio.read(dma_capture_regmap.RegMap.FIFO_WR_PTR_ADDR)))
print("DMA FIFO Read Pointer         : " + str(dma_capture.mmio.read(dma_capture_regmap.RegMap.FIFO_RD_PTR_ADDR)))



# Clear the buffer write address
qpsk_demod.mmio.write(qpsk_regmap.RegMap.WR_RAM_ADDR_CTRL_ADDR,1)

# Configure Sync Word
qpsk_demod.mmio.write(qpsk_regmap.RegMap.SYNC_RESET_ADDR,1)
#qpsk_demod.mmio.write(qpsk_regmap.RegMap.SYNC_WORD_ADDR,0xDEADBEEF)
qpsk_demod.mmio.write(qpsk_regmap.RegMap.SYNC_WORD_ADDR,0x6D75521E)

# Allocate buffers for the input and output signals
in_buffer = allocate(shape=(NumSamples,), dtype=np.int32)


# Copy the samples to the in_buffer
np.copyto(in_buffer,xiq)

# Start HLS QPSK Demodulator
qpsk_demod.mmio.write(qpsk_regmap.RegMap.AP_CONTROL_ADDR,0)
qpsk_demod.mmio.write(qpsk_regmap.RegMap.AP_CONTROL_ADDR,1)

# Trigger DMA Capture 
dma_capture.mmio.write(dma_capture_regmap.RegMap.CAPTURE_STB_ADDR,1)


# Trigger the DMA transfer and wait for the result
import time
start_time = time.time()

# Start DMA Transfer
dma.sendchannel.transfer(in_buffer)
print("Done tranfer in_buffer")

dma.sendchannel.wait()
print("Done send channel wait")


stop_time = time.time()
hw_exec_time = stop_time-start_time
print('DMA Transfer Execution Time   :',hw_exec_time,' sec')

# Free the buffers
in_buffer.close()


print("Sync Lock Indecator           : " + str(qpsk_demod.mmio.read(qpsk_regmap.RegMap.SYNC_LOCK_ADDR)))
print("Symbols in the Buffer         : " + str(qpsk_demod.mmio.read(qpsk_regmap.RegMap.WR_RAM_ADDR)))

# Check AWGN Saturation Flags
awgn_en_reg = awgn.read(awgn_regmap.RegMap.AWGN_ENABLE_ADDR)
print("AWGN I Sat                    : " + str((awgn_en_reg >> 1) & 0x1))
print("AWGN Q Sat                    : " + str((awgn_en_reg >> 2) & 0x1))

print("DMA Capture FIFO Write Pointer: " + str(dma_capture.mmio.read(dma_capture_regmap.RegMap.FIFO_WR_PTR_ADDR)))
print("DMA Capture FIFO Read Pointer : " + str(dma_capture.mmio.read(dma_capture_regmap.RegMap.FIFO_RD_PTR_ADDR)))

NumSamples                    : 8768
NumSymboles                   : 548
AWGN Enable                   : 0
AWGN Gain                     : 0.7998046875
DMA Reset                     : 1
DMA Reset                     : 0
DMA FIFO Depth                : 32768
DMA FIFO Capture Length       : 32
DMA FIFO Write Pointer        : 0
DMA FIFO Read Pointer         : 0
Done tranfer in_buffer
Done send channel wait
DMA Transfer Execution Time   : 0.008796453475952148  sec
Sync Lock Indecator           : 1
Symbols in the Buffer         : 525
AWGN I Sat                    : 0
AWGN Q Sat                    : 0
DMA Capture FIFO Write Pointer: 33
DMA Capture FIFO Read Pointer : 33


### Read QPSK Demodulator Output Bit Buffer

In [ ]:
rd_len = int(qpsk_demod.mmio.read(qpsk_regmap.RegMap.WR_RAM_ADDR)/SamplesPerSym)
print("Number of DWORDs to Read: " + str(rd_len))
dout = []
for i in range(rd_len):
    qpsk_demod.mmio.write(qpsk_regmap.RegMap.RD_RAM_ADDR_ADDR,i)
    dout.append(qpsk_demod.mmio.read(qpsk_regmap.RegMap.RD_RAM_DATA_ADDR))
    
for i in dout:
    print(str(hex(i)))

### Read DMA Data Capture Buffer

In [44]:
# Read the number of samples capture to use for DMA
cap_num = dma_capture.mmio.read(dma_capture_regmap.RegMap.CAPTURE_LENGTH_ADDR)
dma.register_map.MM2S_LENGTH=cap_num*4
print("Num Word Read from DMA Capture: " + str(cap_num))
print("FIFO Write Pointer            : " + str(dma_capture.mmio.read(dma_capture_regmap.RegMap.FIFO_WR_PTR_ADDR)))
print("FIFO Read Pointer             : " + str(dma_capture.mmio.read(dma_capture_regmap.RegMap.FIFO_RD_PTR_ADDR)))
# Allocate DMA Buffer and Read
out_buffer = allocate(shape=(cap_num,), dtype=np.int32)
dma.recvchannel.transfer(out_buffer)
print("Done tranfer out_buffer")
#dma.recvchannel.wait()
print("Done Wait")
sym_iq = out_buffer.astype(np.uint32)
print("Symbols in Buffer             : " + str(len(sym_iq)))
out_buffer.close()

# Re-format Symbols to fixed point values
sym_i = 1-(((sym_iq & 0xFFFF0000) >> 16)/2**15)
sym_q = 1-((sym_iq & 0x0000FFFF)/2**15)
p = figure(width=500, height=500)

# add a circle renderer with a size, color, and alpha
p.scatter(sym_i, sym_q, size=10, marker='asterisk', color="red", alpha=1) 

from bokeh.plotting import figure, show

# show the results
show(p)

print("FIFO Write Pointer            : " + str(dma_capture.mmio.read(dma_capture_regmap.RegMap.FIFO_WR_PTR_ADDR)))
print("FIFO Read Pointer             : " + str(dma_capture.mmio.read(dma_capture_regmap.RegMap.FIFO_RD_PTR_ADDR)))

Num Word Read from DMA Capture: 32
FIFO Write Pointer            : 33
FIFO Read Pointer             : 33
Done tranfer out_buffer
Done Wait
Symbols in Buffer             : 32


FIFO Write Pointer            : 1
FIFO Read Pointer             : 1


#### Debug: Clear QPSK Demodulator Output Buffer Pointer

In [ ]:
print("WR Capture Address: " + str(qpsk_demod.mmio.read(qpsk_regmap.RegMap.WR_RAM_ADDR)))
print("RD Capture Address: " + str(qpsk_demod.mmio.read(qpsk_regmap.RegMap.RD_RAM_ADDR_ADDR)))
qpsk_demod.mmio.write(qpsk_regmap.RegMap.WR_RAM_ADDR_CTRL_ADDR,1)
print("WR Capture Address: " + str(qpsk_demod.mmio.read(qpsk_regmap.RegMap.WR_RAM_ADDR)))
print("SYNC Lock         : " + str(qpsk_demod.mmio.read(qpsk_regmap.RegMap.SYNC_LOCK_ADDR)))
qpsk_demod.mmio.write(qpsk_regmap.RegMap.SYNC_RESET_ADDR,1)
print("SYNC Lock         : " + str(qpsk_demod.mmio.read(qpsk_regmap.RegMap.SYNC_LOCK_ADDR)))

In [45]:
dma.register_map

RegisterMap {
  MM2S_DMACR = Register(RS=0, Reset=0, Keyhole=0, Cyclic_BD_Enable=0, IOC_IrqEn=0, Dly_IrqEn=0, Err_IrqEn=0, IRQThreshold=1, IRQDelay=0),
  MM2S_DMASR = Register(Halted=1, Idle=0, SGIncld=0, DMAIntErr=0, DMASlvErr=0, DMADecErr=0, SGIntErr=0, SGSlvErr=0, SGDecErr=0, IOC_Irq=1, Dly_Irq=0, Err_Irq=0, IRQThresholdSts=0, IRQDelaySts=0),
  MM2S_CURDESC = Register(Current_Descriptor_Pointer=0),
  MM2S_CURDESC_MSB = Register(Current_Descriptor_Pointer=0),
  MM2S_TAILDESC = Register(Tail_Descriptor_Pointer=0),
  MM2S_TAILDESC_MSB = Register(Tail_Descriptor_Pointer=0),
  MM2S_SA = Register(Source_Address=928907264),
  MM2S_SA_MSB = Register(Source_Address=0),
  MM2S_LENGTH = Register(Length=32),
  SG_CTL = Register(SG_CACHE=0, SG_USER=0),
  S2MM_DMACR = Register(RS=0, Reset=0, Keyhole=0, Cyclic_BD_Enable=0, IOC_IrqEn=0, Dly_IrqEn=0, Err_IrqEn=0, IRQThreshold=1, IRQDelay=0),
  S2MM_DMASR = Register(Halted=1, Idle=0, SGIncld=0, DMAIntErr=1, DMASlvErr=0, DMADecErr=0, SGIntErr=0, SGSlv

In [ ]:
dma.register_map.S2MM_LENGTH = 2048
dma.register_map.MM2S_LENGTH = 2048

### Reset the DMA Controller
AXI DMA LogiCORE IP Product Guide: https://docs.amd.com/r/en-US/pg021_axi_dma/MM2S_DMACR-MM2S-DMA-Control-Register-Offset-00h

In [40]:
# Initiate the DMA Reset for both RD/WR
dma.register_map.MM2S_DMACR.Reset = 1
dma.register_map.S2MM_DMACR.Reset = 1

# Re-enable the DMA Channel
dma.register_map.MM2S_DMACR.RS = 1
dma.register_map.S2MM_DMACR.RS = 1

# Complete it again
dma.recvchannel.stop()
dma.sendchannel.stop()
dma.recvchannel.start()
dma.sendchannel.start()

# clear buffers
in_buffer.close()
out_buffer.close()
del in_buffer, out_buffer

In [ ]:
dma.recvchannel.idle
dma.sendchannel.running

In [ ]:
in_buffer.close()
out_buffer.close()